In [1]:
import pandas as pd
import json
import requests
from tabulate import tabulate

USER_DETAILS = json.dumps({
    "username": "AkashAK12",
    "password": "VGF0YUFjZUAxMjM=",
    "browser": "Chrome",
    "browserVersion": "127.0.0.0",
    "device": "Unknown",
    "os": "Windows",
    "osVersion": "windows-10",
    "userAgent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
})
BIDKARO_API = 'https://bidkaro.net/api'


In [2]:
#Lets Login to the website https://bidkaro.ne


data = {
    "username": "AkashAK12",
    "password": "VGF0YUFjZUAxMjM=", 
    "browser": "Chrome",
    "browserVersion": "127.0.0.0",
    "device": "Unknown",
    "os": "Windows",
    "osVersion": "windows-10",
    "userAgent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}

response = requests.post(f"{BIDKARO_API}/Account/CreateToken", json=data)
# set value to token
token = f"Bearer {response.json().get('token')}" 
print(response.json())


{'token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzdWIiOiJQdXNoa2FyLmlpdGtncEBnbWFpbC5jb20iLCJqdGkiOiJkYWQxNmYyZC04YjBkLTQxYWUtYmU4ZS00NDE4YjllNzBiNjMiLCJ1bmlxdWVfbmFtZSI6IkFrYXNoQUsxMiIsIlVzZXJSb2xlcyI6IkJ1eWVyIiwiQXVjdGlvblR5cGVzIjoiQmFuayxJbnN1cmFuY2UiLCJMb2dnZWRJblVzZXJJZCI6IjMyMTc3IiwiVXNlck5hbWUiOiJBa2FzaEFLMTIiLCJJc0xvZ2dlZEluVXNlckhpZGRlbiI6IkZhbHNlIiwiU2VsbGVyQ29kZSI6IiIsIkxvZ2luVHJhY2tpbmdJZCI6IjE2ODM2NTkiLCJQYXNzd29yZENoYWduZWRPbiI6IjcvMjUvMjAyNCAxMTowNDozOSBQTSIsImh0dHA6Ly9zY2hlbWFzLm1pY3Jvc29mdC5jb20vd3MvMjAwOC8wNi9pZGVudGl0eS9jbGFpbXMvcm9sZSI6IkJ1eWVyIiwiZXhwIjoxNzI1Mjk5NDc3LCJpc3MiOiJiaWRrYXJvLm5ldCIsImF1ZCI6IldlYkFwcCJ9.1mZapBTV-QKy2NetuO4G4feGWBRqr09xERPvDkoVnco', 'expiration': '2024-09-02T17:51:17Z', 'initialRoute': '/profile', 'userName': 'AkashAK12', 'userRoles': ['Buyer'], 'auctionTypes': [1, 2], 'userStatus': 4, 'serverTime': '2024-08-03T23:21:17.6772438+05:30', 'passwordChangedDate': '2024-07-25T23:04:39.727', 'menus': [{'route': '/events', 'menuText': 'Events',

In [4]:
# Get list of live auctions on specific filter
# https://bidkaro.net/api/bidDetails/live-auctions?skip=0&pageSize=100&filter=Karnataka&sortOrder=1&auctionEntityType=1&archiveMode=99

FILTER_LOCATION = "Karnataka" # Put empty string for all locations

headers = {
    "Authorization": token,
    "Content-Type": "application/json; charset=utf-8",
    "Accept": "application/json"
}

response = requests.get(f"{BIDKARO_API}/bidDetails/live-auctions?skip=0&pageSize=100&filter={FILTER_LOCATION}&sortOrder=1&auctionEntityType=1&archiveMode=99", headers=headers)

json_response = response.json()

summary_info = {
    "totalItems": json_response.get("totalItems"),
    "pageNumber": json_response.get("pageNumber"),
    "pageSize": json_response.get("pageSize"),
    "totalPages": json_response.get("totalPages")
}
print(f"Total Items: {summary_info['totalItems']}, Page Number: {summary_info['pageNumber']}, Page Size: {summary_info['pageSize']}, Total Pages: {summary_info['totalPages']}")

data_list = json_response.get("list", [])
df = pd.DataFrame(data_list, columns=[
    "auctionId", "auctionIdStr", "categoryName", "closingTime", "startTime", "cug", "guid", "location", "locationId", "sellerCode", "sellerId", "soldAmount", "vehicleCount"
])
df.rename(columns={
    "closingTime": "endTime",
    "cug": "Name of Event",
    "vehicleCount": "vehicleCount"
}, inplace=True)

print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

Total Items: 7, Page Number: 1, Page Size: 100, Total Pages: 1
+-------------+-----------------+----------------+---------------------+---------------------+--------------------------------+--------------------------------------+------------+--------------+--------------+------------+--------------+----------------+
|   auctionId | auctionIdStr    | categoryName   | endTime             | startTime           | Name of Event                  | guid                                 | location   |   locationId | sellerCode   |   sellerId | soldAmount   |   vehicleCount |
+=============+=================+================+=====================+=====================+================================+======================================+============+==============+==============+============+==============+================+
|     1060686 | Bid_Auc_1060686 | 2W             | 2024-08-05T15:29:00 | 2024-08-02T12:50:00 | Muthoot Money Ltd-Karnataka-2W | 20541903-aa3e-47de-9edd-b53343a1b7cc | Karna

In [7]:
# Get list of Particular Bids based on its GUID
AUCTION_GUID = "20541903-aa3e-47de-9edd-b53343a1b7cc" # Put empty string for all locations

response = requests.get(f"{BIDKARO_API}/AuctionDetails/live-auctions?skip=0&pageSize=100&filter=&sortOrder=1&auctionGuid={AUCTION_GUID}", headers=headers)

json_response = response.json()

summary_info = {
    "totalItems": json_response.get("totalItems"),
    "pageNumber": json_response.get("pageNumber"),
    "pageSize": json_response.get("pageSize"),
    "totalPages": json_response.get("totalPages")
}
print(f"Total Items: {summary_info['totalItems']}, Page Number: {summary_info['pageNumber']}, Page Size: {summary_info['pageSize']}, Total Pages: {summary_info['totalPages']}")

data_list = json_response.get("list", [])

for item in data_list:
    image_docs = item.get("imageDocuments", [])
    image_dict = {f"image{i+1}": img["path"] for i, img in enumerate(image_docs)}
    image_json_str = json.dumps(image_dict, indent=2)
    
    # Add image JSON string to the DataFrame
    item["imageDocuments"] = image_json_str

df = pd.DataFrame(data_list, columns=[
    "bidDetailId", "bidIncrement", "bidStatus", "bidderCount", "bidkaroContactNumber", "bidkaroContactPerson", "buyerCommissionRate", "buyerCommissionRateAmount", 'loanAccountNumber',"buyerCommissionRatePercent",'vehicleName','vehicleRegistrationNumber','yardName','yearOfRegistration' ,"chasisNo", "endDateTime", "mainImageURL", "imageDocuments","mileageKms", 'parkingAddress', 'parkingCharges', 'reservePrice', 'startingPrice','sellerCode', 'sellerContactNumber', 'sellerContactPerson', 'sellerId', 'specialClause1', 'specialClause2', 
])

print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))



Total Items: 22, Page Number: 1, Page Size: 100, Total Pages: 1
+---------------+----------------+-------------+---------------+------------------------+------------------------+-----------------------+-----------------------------+----------------------+------------------------------+------------------------------------------+-----------------------------+---------------------------------------+----------------------+-------------------+---------------------+-------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+--------------+---------------------------------------+-------------------------------------------+----------------+-----------------+--------------+-----------------------+-----------------------+------------+--------------------------+-----------------------------------------------------------------+
|   bidDetailId |